# 4.9 Part 01 - Intro to Data Visualization with Python

### This script contains the following points:
- **01. Importing Libraries & Data**
- **02. Wrangle the Data**
- **03. Check the Data**
- **04. Combine Customer Data with Prepared Instacart Data**
- **05. Export the New Dataframe as a Pickle File**

## 01. Importing Libraries & Data

In [1]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Turn Path into a String & Assign Python Shortcut
path = '/Users/mgottlieb/Documents/Achievement 4'

In [3]:
# Import the dataset for this task
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [5]:
# Check the import
df_customers.shape

(206209, 10)

In [6]:
# Check the import
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 02. Wrangle the Data

In this section I will be renaming columns so there is a consistency to the naming convention, right now some include _ and some do not, and some are lower case and some are upper case.\
\
At this point **I will not be dropping any columns** because I am unsure at this specific point in time if any of these columns are indeed erroneous. 

In [9]:
# Print original column names
print("Original column names:", df_customers.columns)

Original column names: Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')


In [12]:
# Assign new column names using camel case for consistency
df_customers.rename(columns={'First Name': 'first_name'}, inplace=True)
df_customers.rename(columns={'Surnam': 'surname'}, inplace=True)
df_customers.rename(columns={'Gender': 'gender'}, inplace=True)
df_customers.rename(columns={'STATE': 'state'}, inplace=True)
df_customers.rename(columns={'Age': 'age'}, inplace=True)

In [13]:
# Print updated column names
print("Updated column names:", df_customers.columns)

Updated column names: Index(['user_id', 'first_name', 'surname', 'gender', 'state', 'age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')


## 03. Check the Data

In [14]:
# View descriptive statistics 
df_customers.describe()

,user_id,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


There does not seem to be anything that looks like it would be unusual in the descriptive analytics.

In [15]:
# Check data types 
df_customers.dtypes

user_id          int64
first_name      object
surname         object
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [18]:
# Checking whether a dataframe contains any mixed-type columns
for col in df_customers.columns.tolist():
    weird = df_customers[col].apply(lambda x: type(x) != type(df_customers[col].iloc[0])).any()
    if weird:
        print(col)

user_id
first_name
age
n_dependants
income


The following columns were listed as containing mixed-type columns, so these will need to be fixed.

In [19]:
# Print out the types of data stored in each of the above identified columnsb
for col in ['user_id', 'first_name', 'age', 'n_dependants', 'income']:
    print(f"Data types in {col}:")
    print(df_customers[col].apply(lambda x: type(x)).value_counts(), '\n')

Data types in user_id:
user_id
<class 'int'>    206209
Name: count, dtype: int64 

Data types in first_name:
first_name
<class 'str'>      194950
<class 'float'>     11259
Name: count, dtype: int64 

Data types in age:
age
<class 'int'>    206209
Name: count, dtype: int64 

Data types in n_dependants:
n_dependants
<class 'int'>    206209
Name: count, dtype: int64 

Data types in income:
income
<class 'int'>    206209
Name: count, dtype: int64 



Based on this information, only the first_name column contains mixed data types, 'str' and 'float'.

In [20]:
# Convert all entries to strings as this makes the most sense for name data
df_customers['first_name'] = df_customers['first_name'].astype('str')

In [21]:
# Checking and verifying the updated column type
print(df_customers['first_name'].dtype)

object


This is now corrected and I can move along in my data check.

In [22]:
# Check for missing values
df_customers.isnull().sum()

user_id         0
first_name      0
surname         0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

There do not appear to be any missing values.

In [23]:
# Check for duplicate rows
df_customer_dupes = df_customers[df_customers.duplicated()]

In [24]:
df_customer_dupes

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income


There are not duplicates, so now that I have completed the data quality and consistency checks, I can move to the next step.

## 04. Combine Customer Data with Prepared Instacart Data

In [25]:
# Import the previosly prepared Instacart data 'orders_products_flags_grouped.pkl'
df_orders_prod_prep = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_flags_grouped.pkl'))

In [26]:
# Check the import
df_orders_prod_prep.shape

(32404859, 25)

In [27]:
# Check the import
df_orders_prod_prep.head()

,order_id,user_id,evaluation_status,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,days_since_prior_order_missing,product_id,add_to_cart_order,...,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,user_avg_price,spending_flag,user_median_days,order_frequency_flag
0,2539329,1,prior,1,2,8,7.0,1,196,1,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
1,2398795,1,prior,2,3,7,15.0,0,196,1,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
2,473747,1,prior,3,3,12,21.0,0,196,1,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
3,2254736,1,prior,4,4,7,29.0,0,196,1,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
4,431534,1,prior,5,4,15,28.0,0,196,1,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer


In [28]:
# Check the 'df_customers' dataframe
df_customers.head()

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In looking at both these datasets, we can see that we can merge them on the 'user_id' column. First I will double check that they are the same datatype.

In [29]:
# Check the datatype of the column to join on
df_orders_prod_prep.dtypes['user_id']

dtype('int64')

In [30]:
# Check the datatype of the column to join on
df_customers.dtypes['user_id']

dtype('int64')

Since these are the same, I can confidently continue on to merge the two dataframes.

In [31]:
# Perform the merge operation on the 'user_id' column. 
# This will be an inner join is used when you want to combine records from both dataframes only where there is a match on the specified key column.
df_combined = pd.merge(df_orders_prod_prep, df_customers, on='user_id', how='inner')

In [32]:
# Check this newly created combined dataframe
df_combined.shape

(32404859, 34)

We can see that the same this has the same number of rows as df_orders_prod_prep (32404859, 25), with the added columns.

In [33]:
df_combined.head(5)

,order_id,user_id,evaluation_status,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,days_since_prior_order_missing,product_id,add_to_cart_order,...,order_frequency_flag,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income
0,2539329,1,prior,1,2,8,7.0,1,196,1,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,prior,2,3,7,15.0,0,196,1,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,prior,3,3,12,21.0,0,196,1,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,prior,4,4,7,29.0,0,196,1,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,prior,5,4,15,28.0,0,196,1,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [34]:
# Check inner join value counts
df_combined['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

The two dataframes now appear successfully combined.

## 05. Export the New Dataframe as a Pickle File

In [35]:
df_combined.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_merged.pkl'))